<center>Hello everyone</center>
I upload my starting notebook, the data is stored on google drive in General access, then continue experimenting with the lens, hair extensions and using EfficientNetB3-B7

In [ ]:
!pip install imutils

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
import gc
import pandas as pd


import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import ZeroPadding2D,UpSampling2D,ThresholdedReLU,Conv2DTranspose, Cropping2D, DepthwiseConv2D,add, Activation, LeakyReLU, Dense, Conv2D, GlobalMaxPooling2D , MaxPooling2D, Flatten,Concatenate, Input, Dropout, BatchNormalization, GlobalAveragePooling2D, SeparableConv2D, AveragePooling2D


from tensorflow.keras.optimizers import RMSprop, Adam, SGD 
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, AUC, BinaryAccuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.activations import softsign
from keras.initializers import Constant
# import gdown
import cv2
import math
from tensorflow.keras.regularizers import l1, l2
from PIL import Image
import shutil
from tqdm import tqdm
import os
from tensorflow.keras.applications import InceptionV3, ResNet101, VGG16
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from skimage import data, img_as_float
from skimage import exposure
from skimage.io import imsave
from imutils import perspective
from imutils import contours
import imutils
from scipy.spatial import distance as dist
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)
import matplotlib.gridspec as gridspec
from sklearn.model_selection import KFold

In [ ]:
TRAIN_DIR = "./train/"
TEST_DIR = "./test/"
HAIR_ADD_IMAGE = False
LINZE = False
CROP = True
CROP_SIZE=15 #15
SHAPE = 256
SHAPE_RESIZE = 150
VAR1 = True
BATCH_SIZE=64
EPOCH = 30
train_dir = "../input/jpeg-melanoma-256x256/train"
test_dir = "../input/jpeg-melanoma-256x256/test"
os.mkdir(TRAIN_DIR)
os.mkdir(TEST_DIR)
DEVICE= "TPU"

In [ ]:
shutil.rmtree(TRAIN_DIR)
shutil.rmtree(TEST_DIR)

In [ ]:
def HAIR_SORRY_REMOVE(image, clip_hist_percent=1):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    kernel = cv2.getStructuringElement(1,(17,17))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _,threshold = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
    final_image = cv2.inpaint(image,threshold,1,cv2.INPAINT_TELEA)
    return (final_image)
def step_assimetry(img):
    img = HAIR_SORRY_REMOVE(img)
#     img = cv2.bilateralFilter(img,50,15,15)
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, thresh =     cv2.threshold(gray,100,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    kernel = np.ones((2,2),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
    kernel = np.ones((2,2),np.uint8)
    dilate = cv2.erode(opening,kernel,iterations=3)
#     dilate = cv2.dilate(opening,kernel,iterations=3)
    blur = dilate
   
#     blur = cv2.blur(opening,(15,15))
    ret, thresh =     cv2.threshold(blur,100,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    contours, hierarchy =     cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
#     images = cv2.drawContours( img, contours, -1, (255,0,0), 3, cv2.LINE_AA, hierarchy, 1 )
#     plt.imshow(images)
    
    delta = 80
#     if len(contours)<3:
#         delta = 40
    S = list()
#     concat = np.concatenate(contours)
# hulls = cv2.convexHull(concat)
    for c in contours:
        box = cv2.minAreaRect(c)
        box = cv2.cv.BoxPoints(box)  if imutils.is_cv2() else cv2.boxPoints( box)
        box = np.array(box, dtype="int")
        box = perspective.order_points( box)
        center = img.shape[0]/2
       
        ( tl, tr, br, bl) = box
        (centerX, centerY) = midpoint( tl, br)
        (midlx, midly) = midpoint( tl, bl)
#         print(midlx,"---")
        if (tl[0]>0 and tr[0]<250 and br[1]<255 and centerX>center-delta and centerX<center+delta and centerY>center-delta and centerY<center+delta) or (midlx>0 and br[0]<252 and centerX>center-delta and centerX<center+delta  and centerY>center-delta and centerY<center+delta):
#             if (centerX>center-delta and centerX<center+delta) and (centerY>center-delta and centerY<center+delta):
                S.append(c)
    if len(S)>0:
        cont = np.vstack([S[i] for i in range(len(S))])
        hull = cv2.convexHull(cont)
        S=hull
    
#     print([S[i] for i in len(S)])
    return S

In [ ]:
COUNTLEN = 10

In [ ]:
def get_Assimetry2(im):
    example = [(105, 24), (78, 84), (93, 90), (105, 112), (110, 148), (93, 179), (76, 228), (44, 225), (112, 255), (184, 252), (217, 255), (255, 224), (255, 123), (207, 110), (209, 56), (153, 78)] 
    result = im.copy()
    orig2 = im.copy()
    img = im.copy()
#     img = segmentation_color3(img.copy())
   
    S = step_assimetry(img)
    shape_orig = orig2.shape[0]
    if len(S)>0:        
#         cnt = max(S, key=cv2.contourArea)
        cnt = S
    else:
        return "err", result, 200, 200
    box = cv2.minAreaRect(cnt)
    box = cv2.cv.BoxPoints(box)  if imutils.is_cv2() else cv2.boxPoints( box)
    box = np.array(box, dtype="int")
    box = perspective.order_points( box)
    orig = cv2.drawContours(result, [box.astype("int")] , -1 , ( 0 , 255 , 0 ) , 2)
    for ( x, y)  in box:
        cv2.circle(result, (int(x), int(y)) , 5 , ( 0 , 0 , 255 ) , -1)
    ( tl, tr, br, bl) = box
    d1 = dist.euclidean(tl, tr)
    d2 = dist.euclidean(tl, bl)
    ( centerXX, centerYY) = midpoint( tl, br)
    (midlx, midly) = midpoint( tl, bl)
#     print(int(midly),int(midlx))
    cv2.circle(result,(int(midlx), int(midly)),15,(255,45,12),2)
    start = (int(centerXX), int( centerYY))
    count_len = COUNTLEN
    shape = cnt.shape[0]
    run = int(shape/count_len)
    if shape<count_len:
        run = 1
    LENGTH = list()
    sss = []
    previos = start
    
    for i,r in enumerate(range(0,shape,run)):
        if i<count_len:
            end = (int(cnt[r][0][0]),int(cnt[r][0][1]))
            dA = dist.euclidean(start, end)
            
            cv2.line(result, start, end,(0, 0, 0), 2)
            previos = end
            
            
            
            LENGTH.append(dA)
        else:
            break
    max_LENGTN = max(LENGTH)
    min_LENGTN = min(LENGTH)
    if len(LENGTH)<count_len:
        razn = count_len - len(LENGTH)
        for i in range(razn):
            LENGTH.append(0)
    if max_LENGTN<28 or min_LENGTN<12:
        cv2.circle(result,(int(midlx), int(midly)),20,(255,45,12),9)
        return 250, result, d1,d2
#         max_LENGTN=55#80
#     max_LENGTN+=10#15
    top_crop = centerYY-max_LENGTN
    bottom_crop = centerYY+max_LENGTN
    left_crop = centerXX-max_LENGTN
    right_crop = centerXX+max_LENGTN
    if top_crop<0:
        top_crop=0
    if bottom_crop>256:
        bottom_crop=256
    if left_crop<0:
        left_crop=0
    if right_crop>256:
        right_crop=256
#     result = result[int(top_crop):int(bottom_crop),int(left_crop):int(right_crop),:]
#     result = cv2.bilateralFilter(result,9,100,100)
#     result = segmentation_color3(result)
#     print (sss)
    return LENGTH, result, d1,d2 #return length border

In [ ]:
train = pd.read_csv("../input/jpeg-melanoma-256x256/train.csv")
test =  pd.read_csv("../input/jpeg-melanoma-256x256/test.csv")

In [ ]:
train

In [ ]:
def constructing_features(DIR,DATAFRAME,save_dir):
    DATAFRAME["pred_pred1"] = 0
    DATAFRAME["pred_pred2"] = 0
    for i, image_name in enumerate(tqdm(DATAFRAME["image_name"])):
        
        im3 = cv2.imread(DIR+"/"+image_name+".jpg")
        LENGTH, crop_image,d1,d2 = get_Assimetry2(im3)
        if LENGTH=="err":
            DATAFRAME.loc[i,"pred_pred1"] = 1
        if d1<100 and d2<100 and d1>26 and d2>26 and abs(d1-d2)<6:
            DATAFRAME.loc[i,"pred_pred2"] = 1
        crop_image = im3
        crop_image = cv2.resize(crop_image, (SHAPE_RESIZE,SHAPE_RESIZE), interpolation = cv2.INTER_AREA)
        mask = np.full((SHAPE_RESIZE, SHAPE_RESIZE), 0, dtype=np.uint8)
       
    
        cv2.circle(mask, (SHAPE_RESIZE//2, SHAPE_RESIZE//2) , 90 , ( 255 , 0 , 0 ) , -1)
        crop_image= cv2.bitwise_or(crop_image, crop_image, mask=mask)
        anatom_site_general_challenge = DATAFRAME.loc[i,"anatom_site_general_challenge"]
        if anatom_site_general_challenge=="torso":
            cv2.circle(crop_image,(SHAPE_RESIZE-20, SHAPE_RESIZE-20),10,(255,45,12),3)
        if anatom_site_general_challenge=="lower extremity":
            cv2.circle(crop_image,(SHAPE_RESIZE-20, SHAPE_RESIZE-20),10,(0,255,0),3)
            cv2.circle(crop_image,(SHAPE_RESIZE-40, SHAPE_RESIZE-20),10,(0,255,0),3)
        if anatom_site_general_challenge=="head/neck":
            cv2.circle(crop_image,(SHAPE_RESIZE-20, 20),10,(0,0,255),3)
            cv2.circle(crop_image,(SHAPE_RESIZE-40, 20),10,(0,0,255),3)
            cv2.circle(crop_image,(SHAPE_RESIZE-60, 20),10,(0,0,255),3)
        if anatom_site_general_challenge=="upper extremity":
            cv2.rectangle(crop_image, (30,20), (50,40), (90,120,255), 3)
        if anatom_site_general_challenge=="palms/soles":
            cv2.rectangle(crop_image, (30,20), (50,40), (90,120,255), 3)
            cv2.rectangle(crop_image, (40,20), (60,40), (90,120,255), 3)
        if anatom_site_general_challenge=="oral/genital":
            cv2.rectangle(crop_image, (30,SHAPE_RESIZE-50), (50,SHAPE_RESIZE-30), (90,120,56), 3)
        
        crop_image = cv2.cvtColor(crop_image, cv2.COLOR_BGR2RGB)
        
#         colors = segmintation_color(crop_image)

        plt.imsave(save_dir+image_name+".jpg", crop_image) 
constructing_features(train_dir,train,TRAIN_DIR)

In [ ]:
train[(train.pred_pred==1) & (train.target==1)]

Adding data from google disk, here you can experiment with the lens, hair extensions using opencv

<center><h1>TEST IMAGE</h1></center>

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=6, figsize=(15,15), gridspec_kw={'wspace':0.1, 'hspace':0})
j=0
ran = 9000
for i in range(ran+6):
  if i>ran-1:
    img = cv2.imread(test_dir+test_image_name_arr[i]+".jpg")
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    ax[j].imshow(img)
    ax[j].axis("off")
    j+=1

<center><h1>TRAIN IMAGE<h1></center>

In [ ]:
test_image_name_arr = train[train.target==0]["image_name"].values
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(15,15), gridspec_kw={'wspace':0.1, 'hspace':0})
j=0
ran = 0
for i in range(ran+4):
  if i>ran-1:
    img = cv2.imread(TRAIN_DIR+test_image_name_arr[i]+".jpg")
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    ax[j].imshow(img)
    ax[j].axis("off")
    j+=1

In [ ]:
img_input = Input(shape=(256,256,3))
con = MaxPooling2D((10, 10), strides=2, padding="same")(img_input)
con = tf.keras.layers.UpSampling2D(size=(2, 2),interpolation="nearest")(con)
con1 =  Conv2D(3,(2,2), activation="relu", kernel_initializer=tf.keras.initializers.Ones())(con)

# con2 =  Conv2D(3,(2,2), activation="elu")(con)

# maxed = tf.keras.layers.Subtract()([con1, con2])
# tf.keras.layers.Add
model2 = Model(inputs = [img_input], outputs=[con1])

inp = model2.layers[0].name
conv = model2.layers[3].name
model2.summary()

In [ ]:
import keras.backend as K
image_name_arr = train[train.target==0]["image_name"].values
print(image_name_arr[1])
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,15), gridspec_kw={'wspace':0.1, 'hspace':0})
j=0
ran = 1234
for i in range(1):
    img2 = cv2.imread(train_dir+"/"+image_name_arr[12678]+".jpg")
    _, img3,_,_ = get_Assimetry2(img2)
    img2 = cv2.cvtColor(img3,cv2.COLOR_BGR2RGB)
    img2 = cv2.resize(img2, (256,256), interpolation = cv2.INTER_AREA)
    tenzor = np.expand_dims(img2,axis=0)
    func = K.function([model2.get_layer(inp).input], model2.get_layer(conv).output)
    conv_output = func([tenzor])  # numpy array
#     print(conv_output.shape)
    from keras.preprocessing import image as ik
    image_aug = ik.array_to_img(conv_output[0])
    image_aug = np.array(image_aug)
#     print(image_aug)
# #     plt.imshow(image_aug)
#     print(image_aug)
    ax[0].imshow(image_aug)
    ax[1].imshow(img2)
    j+=1

In [ ]:
# train.loc[train["sex"].isnull(),["sex"]] = "male"
# train.loc[train["age_approx"].isnull(),["age_approx"]] = 50
# train.loc[train["anatom_site_general_challenge"].isnull(),["anatom_site_general_challenge"]] = "torso"
# train["split"] = 0

# train.loc[train["age_approx"]<=40,["split"]] = 1
# train.loc[(train["age_approx"]>40) & (train["age_approx"]<=76),["split"]] = 2
# train.loc[train["age_approx"]>76,["split"]] = 3

In [ ]:
# train["age_approx"]
# patient_id = LabelEncoder()
# sex = LabelEncoder()
# # age_approx = LabelEncoder()
# anatom_site_general_challenge = LabelEncoder()

# patient_id.fit(train["patient_id"].unique())
# sex.fit(train["sex"].unique())
# # age_approx.fit(train["age_approx"].unique())
# anatom_site_general_challenge.fit(train["anatom_site_general_challenge"].unique())

# train["patient_id"] = patient_id.transform(train["patient_id"])
# train["sex"] = sex.transform(train["sex"])
# # train["age_approx"] = age_approx.transform(train["age_approx"])
# train["anatom_site_general_challenge"] = anatom_site_general_challenge.transform(train["anatom_site_general_challenge"])

In [ ]:
METRICS = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'), 
      BinaryAccuracy(name='accuracy'),
      AUC(name='auc')
]

In [ ]:
!pip install efficientnet
import efficientnet.tfkeras as efn

In [ ]:
EPOCHS=15

In [ ]:
from tensorflow.keras import backend as K
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * 1

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
rng = [i for i in range(25 if EPOCHS<25 else EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

In [ ]:
train1 = train[(train.pred_pred1==0) & (train.pred_pred2==0)].copy()
train2 = train[train.pred_pred1==1].copy()
train3 = train[train.pred_pred2==1].copy()
train1.reset_index(inplace=True, drop=True)
train2.reset_index(inplace=True, drop=True)
train3.reset_index(inplace=True, drop=True)

In [ ]:
train2

In [ ]:
train_split = 0
train_val_split = 0
SHAPE_RESIZE = 256
BATCH_SIZE = 32
# split = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=124)
# # for train_index, test_index in split.split(train,train[["split","sex","target"]]):
# f = 0
# for train_index, test_index in split.split(train,train["target"]):
cv = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in cv.split(train1['target']):
    train_split = train1.loc[train_index].copy()
    train_val_split = train1.loc[test_index].copy()
#     print(train_split.shape)
    train_split.reset_index(inplace=True, drop=True)
    train_val_split.reset_index(inplace=True, drop=True)
    train_split["image_name"] = train_split["image_name"].apply(lambda x: "%s.%s"%(x,"jpg"))
    train_val_split["image_name"] = train_val_split["image_name"].apply(lambda x: "%s.%s"%(x,"jpg")) 
    train_split["target"] = train_split["target"].astype("str")
    train_val_split["target"] = train_val_split["target"].astype("str")
    
    def pre_func(image):
        mask = np.full((SHAPE, SHAPE), 0, dtype=np.uint8)
        cv2.circle(mask, (SHAPE_RESIZE//2, SHAPE_RESIZE//2) , 130 , ( 255 , 0 , 0 ) , -1)
        image= cv2.bitwise_or(image, image, mask=mask)
        return image
    val_generator=0
    train_generator=0
    model=0
    conv_base=0
    train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=30,
#                                    brightness_range=[0.9,1.5],
            
#                                    zoom_range=[0.5,0.8],
                                   preprocessing_function=pre_func
                                   )
    val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function=pre_func
                                )

    val_generator  = val_datagen.flow_from_dataframe(dataframe=train_val_split, directory=train_dir+"/",
                                             x_col='image_name',
                                             y_col="target",
                                             class_mode='binary',
                                             batch_size=BATCH_SIZE,
                                             target_size=(SHAPE_RESIZE, SHAPE_RESIZE),
                                             seed=42)

    train_generator  = train_datagen.flow_from_dataframe(dataframe=train_split, directory=train_dir+"/",
                                             x_col='image_name',
                                             y_col="target",
                                             class_mode='binary',
                                             batch_size=BATCH_SIZE,
                                             target_size=(SHAPE_RESIZE, SHAPE_RESIZE),
                                             seed=23)
    img_input = Input(shape=(SHAPE_RESIZE,SHAPE_RESIZE,3))
    conv_base = efn.EfficientNetB3(weights="imagenet", 
                               include_top=False, 
                               input_shape=(SHAPE_RESIZE,SHAPE_RESIZE,3))
    x = conv_base(img_input)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     x = Dense(1024, activation="relu")(x)
# #     x = BatchNormalization()(x)
#     x =Dropout(0.2)(x)
#     x = Dense(512, activation="relu")(x)
#     x = BatchNormalization()(x)
#     x =Dropout(0.2)(x)
#     x = Dense(212, activation="tanh")(x)
#     x = BatchNormalization()(x)
#     x =Dropout(0.2)(x)
#     x = Dense(112, activation="tanh")(x)
#     x = BatchNormalization()(x)
#     x =Dropout(0.2)(x)
#     for layer in conv_base.layers[:]:
#         layer.trainable = True
    out = Dense(1, activation="sigmoid")(x)
    model = Model(inputs = [img_input], outputs=[out])
    model.compile(loss = [binary_focal_loss(gamma = 2.0, alpha = 0.80)],
              optimizer=Adam(learning_rate=1e-4), metrics=METRICS)
    CW = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_split["target"]),
                                                 train_split["target"])
    clases = [0,1]
    class_weights = dict(zip(clases,CW))
    train_generator.reset()
    history = model.fit_generator(train_generator,
                              steps_per_epoch=(train_split.shape[0]//BATCH_SIZE),
                              epochs=15,
#                                 callbacks=[lr_schedule],
                              validation_data = val_generator,
                              validation_steps=(train_val_split.shape[0]//BATCH_SIZE),
#                               class_weight=class_weights
                              )



In [ ]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
train_split = 0
train_val_split = 0
for train_index, test_index in cv.split(train['target']):
    train_split = train.loc[train_index].copy()
    train_val_split = train.loc[test_index].copy()
#     print(train_split.shape)
    train_split.reset_index(inplace=True, drop=True)
    train_val_split.reset_index(inplace=True, drop=True)
    train_split["image_name"] = train_split["image_name"].apply(lambda x: "%s.%s"%(x,"jpg"))
    train_val_split["image_name"] = train_val_split["image_name"].apply(lambda x: "%s.%s"%(x,"jpg")) 
    train_split["target"] = train_split["target"].astype("str")
    train_val_split["target"] = train_val_split["target"].astype("str")

In [ ]:
state = 32
train_split_2_1 = train_split[train_split.target=="1"].copy()
train_split_2_0 = train_split[train_split.target=="0"].copy()
train_split_2_0_V1 = train_split_2_0.sample(frac =.50, random_state=state).copy()
train_split_2_0_V2 = train_split_2_0.sample(frac =.50, random_state=state).copy()
train_split_2_0_V1_1 = train_split_2_0_V1.sample(frac =.50, random_state=state).copy()
train_split_2_0_V2_2 = train_split_2_0_V2.sample(frac =.50, random_state=state).copy()
train_split_2_0_V1_1_1 = train_split_2_0_V1_1.sample(frac =.50, random_state=state).copy()
train_split_2_0_V2_2_2 = train_split_2_0_V2_2.sample(frac =.50, random_state=state).copy()
train_split_2_0_V1_1_1_1 = train_split_2_0_V1_1_1.sample(frac =.60, random_state=state).copy()
train_split_2_0_V2_2_2_2 = train_split_2_0_V2_2_2.sample(frac =.60, random_state=state).copy()
train_split_2_0_V2_2_2 = train_split_2_0_V2_2.sample(frac =.50, random_state=state).copy()
train_split_2_0_V1_1_1_1 = train_split_2_0_V1_1_1.sample(frac =.60, random_state=state).copy()
train_split_2_0_V2_2_2_2 = train_split_2_0_V2_2_2.sample(frac =.60, random_state=state).copy()
# train_split_2_0_V1_1_1_1_1 = train_split_2_0_V1_1_1_1.sample(frac =.50).copy()

train_split_4_1 = train_val_split[train_val_split.target=="1"].sample(frac =.70, random_state=32)
train_split_4_2 = train_val_split[train_val_split.target=="1"].sample(frac =.30, random_state=45)
train_split_4_3 = train_val_split[train_val_split.target=="1"].sample(frac =.10, random_state=12)
podviborka = 0

In [ ]:
podviborka = train_split_2_0_V1.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1, ignore_index = True).copy()
podviborka = podviborka.append(train_split_2_1.sample(frac =.18, random_state=12), ignore_index = True).copy()
CW = class_weight.compute_class_weight('balanced',
                                                 np.unique(podviborka["target"]),
                                                 podviborka["target"])
clases = [0,1]
class_weights = dict(zip(clases,CW))
podviborka["target"] = podviborka["target"].astype("str")
melonima = podviborka[podviborka.target=="1"].shape[0]
nevus = podviborka[podviborka.target=="0"].shape[0]
podviborka = podviborka.sample(frac=1).reset_index(drop=True)
print(melonima,nevus,class_weights)


In [ ]:
train[train.target==1]

In [ ]:
# train_val_split = train_val_split.append(train_split_2_1, ignore_index = True).copy()
melonima_v = train_val_split[train_val_split.target=="1"].shape[0]
nevus_v = train_val_split[train_val_split.target=="0"].shape[0]
print(melonima_v,nevus_v)


In [ ]:
 def pre_func(image):
        mask = np.full((SHAPE, SHAPE), 0, dtype=np.uint8)
        print(image.filename)
    
        cv2.circle(mask, (SHAPE//2, SHAPE//2) , 130 , ( 255 , 0 , 0 ) , -1)
        image= cv2.bitwise_or(image, image, mask=mask)
        return image

train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
#                                    rotation_range=10,
#                                    brightness_range=[0.9,1.5],
            
#                                    zoom_range=[0.5,0.8],
                                   preprocessing_function=pre_func
                                   )
val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function=pre_func
                                )

val_generator  = val_datagen.flow_from_dataframe(dataframe=train_val_split, directory=train_dir+"/",
                                             x_col='image_name',
                                             y_col="target",
                                             class_mode='binary',
                                             batch_size=BATCH_SIZE,
                                             target_size=(SHAPE_RESIZE, SHAPE_RESIZE),
                                             seed=42)

train_generator  = train_datagen.flow_from_dataframe(dataframe=train_split, directory=train_dir+"/",
                                             x_col='image_name',
                                             y_col="target",
                                             class_mode='binary',
                                             batch_size=BATCH_SIZE,
                                             target_size=(SHAPE_RESIZE, SHAPE_RESIZE),
                                             seed=23)

In [ ]:
# podviborka[list(podviborka.columns)] = podviborka[list(podviborka.columns)].astype("str")
# train_val_split[list(train_val_split.columns)] = train_val_split[list(train_val_split.columns)].astype("str")

In [ ]:
# train_val_c = train_val_split.copy()
# podviborka_c = podviborka.copy()
# # train_val_c.drop(["patient_id","sex","anatom_site_general_challenge","diagnosis","benign_malignant"], axis=1, inplace=True)
# # podviborka_c.drop(["patient_id","sex","anatom_site_general_challenge","diagnosis","benign_malignant"] , axis=1, inplace=True)
# # train_val_c = train_val_c[["image_name","age_approx","veil","globuli","target"]]
# # podviborka_c = podviborka_c[["image_name","age_approx","veil","globuli","target"]]

In [ ]:
from keras.preprocessing import image as ik
image_name_arr = train["image_name"].values
image_name_arr = image_name_arr[1934]
im = Image.open(train_dir+"/"+image_name_arr+".jpg")

tenzor = np.expand_dims(im,axis=0)
j=0
fig, ax = plt.subplots(nrows=1, ncols=6, figsize=(15,15), gridspec_kw={'wspace':0.1, 'hspace':0})
for batch in train_datagen.flow(tenzor, batch_size=1):
        image_aug = ik.array_to_img(batch[0])
        image_aug = np.array(image_aug)
        ax[j].imshow(image_aug)
        ax[j].axis("off")
        j+=1
        if j%6==0:
          break

In [ ]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=90,
#                                    horizontal_flip=True,
#                                    vertical_flip=True,
# #                                    brightness_range=[0.5,1.5],
# #                                    preprocessing_function=preprocess_input
# #                                    zoom_range=0.2,
# #                                    shear_range=0.2
#                                    )
# val_datagen = ImageDataGenerator(rescale=1./255)

# val_generator  = val_datagen.flow_from_dataframe(dataframe=train_val_split, directory=train_dir,
#                                              x_col='image_name',
#                                              y_col=list(train_val_c.columns),
#                                              class_mode='raw',
#                                              batch_size=BATCH_SIZE,
#                                              target_size=(SHAPE, SHAPE),
#                                              seed=7)

# train_generator  = train_datagen.flow_from_dataframe(dataframe=podviborka, directory=train_dir,
#                                              x_col='image_name',
#                                              y_col=list(podviborka_c.columns),
#                                               class_mode='raw',
#                                              batch_size=BATCH_SIZE,
#                                              target_size=(SHAPE, SHAPE),
#                                              seed=7)

# table1 = podviborka[["age_approx","veil","globuli"]].copy()
# table2 = train_val_split[["age_approx","veil","globuli"]].copy()
# def generator_two_inputs(image_generator):

#     i=0
#     count = 0
#     while True:
#         data = image_generator.next()    
#         label = data[1][:,4:5].astype(int)
#         image = data[0]
#         table_data = data[1][:,1:4].astype(int)
# #         print(label)
#         yield [image,table_data], label


In [ ]:
LR_START = 0.000005
LR_MAX = 0.00000725 * 1
LR_MIN = 0.000005
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 4
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

rng = [i for i in range(EPOCH)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
# pos = podviborka[podviborka.target=="1"].shape[0]
# neg = podviborka[podviborka.target=="0"].shape[0]
# initial_bias = np.log([pos/neg])
# output_bias = tf.keras.initializers.Constant(initial_bias)

In [ ]:
SHAPE

In [ ]:
from tensorflow.keras.applications import ResNet101

In [ ]:
SHAPE

In [ ]:
img_input = Input(shape=(SHAPE,SHAPE,3))
CROP=0
# crop = Cropping2D(cropping=((CROP, CROP), (CROP, CROP)), input_shape=(SHAPE,SHAPE,3))(img_input)
conv_base = efn.EfficientNetB3(weights="imagenet", 
                               include_top=False, 
                               input_shape=(SHAPE-CROP,SHAPE-CROP,3))
x = conv_base(img_input)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(512, activation="tanh")(x)
# x = BatchNormalization()(x)
x =Dropout(0.2)(x)
x = Dense(312, activation="tanh")(x)
# x = BatchNormalization()(x)
x =Dropout(0.2)(x)
x = Dense(212, activation="tanh")(x)
# x = BatchNormalization()(x)
x =Dropout(0.2)(x)
x = Dense(112, activation="tanh")(x)
# x = BatchNormalization()(x)
x =Dropout(0.2)(x)
for layer in conv_base.layers[:]:
    layer.trainable = True
out = Dense(1, activation="sigmoid")(x)
model = Model(inputs = [img_input], outputs=[out])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.07),
              optimizer=Adam(
learning_rate=1e-4), metrics=METRICS)
model.summary()

In [ ]:
# conv_base = efn.EfficientNetB3(weights="imagenet", include_top=False, input_shape=(SHAPE,SHAPE,3), pooling="avg")
# # ResNet18, preprocess_input = Classifiers.get('seresnet152')
# # conv_base = ResNet18((SHAPE, SHAPE, 3), weights='imagenet')
# # # conv_base = efn.EfficientNetB2(weights="imagenet", include_top=False, input_shape=(SHAPE,SHAPE,3))
# conv_base.trainable = True



# img_input = Input(shape=(SHAPE,SHAPE,3))
# table_input = Input(shape=(3,))

# x = conv_base(img_input)
# x = Dense(512, activation="tanh")(x)
# # x = BatchNormalization()(x)
# x =Dropout(0.2)(x)
# x = Dense(312, activation="tanh")(x)
# # x = BatchNormalization()(x)
# x =Dropout(0.2)(x)
# x = Dense(212, activation="tanh")(x)
# # x = BatchNormalization()(x)
# x =Dropout(0.2)(x)
# x = Dense(112, activation="tanh")(x)
# # x = BatchNormalization()(x)
# x =Dropout(0.2)(x)

# t = Dense(300, activation="elu")(table_input)
# t = BatchNormalization()(t)
# t = Dense(100, activation="elu")(t)

# concat = tf.keras.layers.concatenate([t,x])
# # out = Dense(1, activation="sigmoid", bias_initializer=output_bias )(concat)
# out = Dense(1, activation="sigmoid")(concat)

# model = Model(inputs = [img_input,table_input], outputs=[out])
for layer in conv_base.layers[1:]:

#     layer.trainable = True
# model.compile(loss="binary_crossentropy", optimizer=Adam(
#     learning_rate=1e-4), metrics=METRICS)
# model.summary()

In [ ]:
import keras
class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch >6:  # or save after some epoch, each k-th epoch etc.
            self.model.save("model_{}.hd5".format(epoch))
saver = CustomSaver()

In [ ]:
class_weights

In [ ]:
# train_generator.reset()
# val_generator.reset()
# train_generator2 = generator_two_inputs(train_generator)
# val_generator2 = generator_two_inputs(val_generator)
# history=0
# history = model.fit_generator(train_generator2,
#                               steps_per_epoch=1+(podviborka.shape[0]//BATCH_SIZE),
#                               epochs=1,
                      
                              
                              
                              
# #                               callbacks=[lr_callback],
#                               validation_data = val_generator2,
#                               validation_steps=1+(train_val_split.shape[0]//BATCH_SIZE),
#                               class_weight=class_weights
#                               )

In [ ]:
model.save('08943.h5')

In [ ]:
os.mkdir(TEST_DIR)

In [ ]:
test2 = test.copy()
# test2.drop(["patient_id","sex","anatom_site_general_challenge"], axis=1, inplace=True)
# test2 = test[["image_name","age_approx","veil","globuli"]]
# podviborka_c = podviborka_c[["image_name","age_approx","veil","globuli","target"]]

In [ ]:
sss = []
constructing_features(test_dir,test2,TEST_DIR)

In [ ]:
len(sss)

In [ ]:
test2["image_name"] = test2["image_name"].apply(lambda x: "%s.%s"%(x,"jpg"))

In [ ]:
test2

In [ ]:
test_image_name_arr = test["image_name"].values
fig, ax = plt.subplots(nrows=1, ncols=6, figsize=(15,15), gridspec_kw={'wspace':0.1, 'hspace':0})
j=0
ran = 567
for i in range(ran+6):
  if i>ran-1:
    img = cv2.imread(test_dir+"/"+test_image_name_arr[i]+".jpg")
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    ax[j].imshow(img)
    ax[j].axis("off")
    j+=1

In [ ]:
test_image_name_arr = test["image_name"].values
fig, ax = plt.subplots(nrows=1, ncols=6, figsize=(15,15), gridspec_kw={'wspace':0.1, 'hspace':0})
j=0
ran = 567
for i in range(ran+6):
  if i>ran-1:
    img = cv2.imread(TEST_DIR+test_image_name_arr[i]+".jpg")
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    ax[j].imshow(img)
    ax[j].axis("off")
    j+=1

In [ ]:
# def generator_two_inputs2(image_generator):

#     i=0
#     count = 0
#     while True:
#         data = image_generator.next()    
#         image = data[0]
#         table_data = data[1][:,1:4].astype(int)
#         count+=1
#         if count>test2.shape[0]:
#             break
#         yield [image,table_data]
        

In [ ]:
test_generator = ImageDataGenerator(rescale=1./255)
pred_test = test_generator.flow_from_dataframe(dataframe=test2, directory=TEST_DIR,
                                             x_col='image_name',
                                             class_mode=None,
                                             shuffle=False, 
                                             batch_size=1,
                                             target_size=(SHAPE, SHAPE)
                                              )
pred_test.reset()
pred=model.predict_generator(pred_test, verbose=1, )

In [ ]:
test =  pd.read_csv("../input/jpeg-melanoma-256x256/test.csv")

In [ ]:
test["target"] = pred

In [ ]:
submis =  test[["image_name","target"]]

In [ ]:
# test_generator = ImageDataGenerator(rescale=1./255)
# pred_test = test_generator.flow_from_dataframe(dataframe=test2, directory=TEST_DIR,
#                                              x_col='image_name',
#                                              y_col=list(test2.columns),
#                                              class_mode='raw',
#                                              shuffle=False, 
#                                              batch_size=1,
#                                              target_size=(SHAPE, SHAPE)
#                                              )
# pred_test.reset()
# test_generator2 = generator_two_inputs2(pred_test)

# pred=model.predict_generator(test_generator2, verbose=1, )
# # test.reset_index(inplace=True, drop= True)
# test2 = test2.join(pd.DataFrame(pred))
# s = list(test2.columns)[:-1]
# s.append("target")
# test2.columns = s


# test2.drop(["patient_id","sex","age_approx","anatom_site_general_challenge"], inplace=True, axis=1)

In [ ]:
pred_test.filenames

In [ ]:
max(submis["target"])

In [ ]:
submis.to_csv("sssss.csv", index=False, line_terminator="\n")